Importing Libraries

In [ ]:
import pandas as pd
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

Loading the fresh csv file from processed data folder

In [ ]:
def get_processed_dir():
    project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))  
    local_path = os.path.join(project_root, "data", "processed_data")
    # Detects path based on environment (Docker vs. local)
    if os.path.exists("/opt/airflow/data/processed_data"):
        return "/opt/airflow/data/processed_data"  # Docker path
    else:
        return local_path # Local path
    
process_folder = get_processed_dir()
print(process_folder)

files = [ f for f in os.listdir(process_folder) if f.startswith('cleaned_') and 
         f.endswith('.csv')]
files.sort(reverse=True)
latest_file = files[0]
latest_path_input = os.path.join(process_folder,latest_file)
df = pd.read_csv(latest_path_input)

print("laoding file ", latest_file)
df.head(100)

Check for duplicate records


In [ ]:
duplicate_count = df.duplicated().sum()
print("Number of duplicated rows", duplicate_count)

Check for null/missing values

In [ ]:
df.isnull().sum()


Checking the data frame column datatype information

In [ ]:
df.info()

Changing the Date column datatype to datetime datatype

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df.info()

In [ ]:
df.head(20)

Added a new column computed total to verify whether the Total sales amount is correctly calculated or not.

In [ ]:
df['computed_total'] = df['Quantity'] * df['Price per Unit']


error = df[df['computed_total'] != df['Total Amount']]
print("Error in number of records", len(error))
df.head(20)


Started some grouping w.r.t Product category 

In [ ]:
product_category_summary = df.groupby('Product Category').agg(
    {
    'Total Amount': 'sum',
    'Quantity': 'sum'
    }
).reset_index()
product_category_summary.columns = ['Product Category', 'Total Sales Amount', 'Total Quantity']
product_category_summary

Minimum selling Product category with amount

In [ ]:
min_category = product_category_summary.loc[product_category_summary['Total Sales Amount'].idxmin()]
print(f"Minimum selling category of item with it total sales : {min_category['Product Category']} → ₹{min_category['Total Sales Amount']}")

Maximum selling Product category with amount

In [ ]:
max_category = product_category_summary.loc[product_category_summary['Total Sales Amount'].idxmax()]
print(f"Max selling category of item with it total sales : {max_category['Product Category']} → ₹{max_category['Total Sales Amount']}")

Grouped according to Product Category and Gender to understand Gender-wise bifurcation in each Product Category.

In [ ]:
category_gender_sales = df.groupby(['Product Category', 'Gender']).agg({
    'Total Amount': 'sum'
}).reset_index()
category_gender_sales

Simple Re-structuring of the dataframe, Product category is treated as index.

In [ ]:
pivot_sales = category_gender_sales.pivot(index='Product Category', columns='Gender', values='Total Amount').fillna(0)

pivot_sales['Total Sales'] = pivot_sales.sum(axis=1)

pivot_sales

Graph Visualization

In [ ]:

plt.figure(figsize=(10, 6))
sns.barplot(data=product_category_summary, x='Product Category', y='Total Sales Amount')

plt.title("Total Sales by Product Category")
plt.ylabel("Total Sales Amount in rupee ")
plt.xlabel("Product Category")
plt.show()

In [ ]:

gender_category_sales = df.groupby(['Gender', 'Product Category'])['Total Amount'].sum().reset_index()


genders = gender_category_sales['Gender'].unique()

for gender in genders:
    plt.figure(figsize=(6, 6))
    data = gender_category_sales[gender_category_sales['Gender'] == gender]
    plt.pie(
        data['Total Amount'],
        labels=data['Product Category'],
        autopct='%1.1f%%',
        startangle=140,
        colors=sns.color_palette('pastel')
    )
    plt.title(f"Sales Distribution by Product Category ({gender})")
    plt.axis('equal')
    plt.tight_layout()
    plt.show()


Saving the tranformed and aggregated data in separate folders

In [ ]:
os.makedirs("../data/transformed", exist_ok=True)

In [ ]:
df.to_csv("../data/transformed/final_cleaned_sales_data.csv", index=False)

In [ ]:
os.makedirs("../data/aggregated", exist_ok=True)

In [ ]:
product_category_summary.to_csv("../data/aggregated/sales_by_category.csv", index=False)

In [ ]:
category_gender_sales.to_csv("../data/aggregated/sales_by_gender.csv", index=False)
